In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.ndimage as ndi
import skimage.filters as fl
import warnings

warnings.filterwarnings('ignore')

In [9]:
from numpy import uint8, int64, float64, array, arange, zeros, zeros_like, ones, mean
from numpy.fft import fft, fft2, ifft, ifft2, fftshift
from math import log2
from scipy.ndimage import convolve, correlate, uniform_filter, gaussian_laplace, gaussian_filter, generic_filter, minimum_filter, maximum_filter, median_filter, rank_filter
from scipy.signal import wiener
from skimage import io, data
from skimage.color import rgb2gray
from skimage.draw import polygon
from skimage.exposure import adjust_gamma, equalize_hist, rescale_intensity
from skimage.filters import threshold_otsu, threshold_isodata, prewitt_h, prewitt_v, prewitt, roberts, sobel, laplace
from skimage.io import imshow
from skimage.transform import rescale
from skimage.util import img_as_ubyte, img_as_float, img_as_bool, random_noise

## numpy

In [7]:
def add(image, c):
    return uint8(np.clip(float64(image) + c, 0, 255))

## matplotlib

In [11]:
def matplot(img, title=None, cmap=None, figsize=None):
    col = len(img)
    
    if figsize is None:
        plt.figure(figsize=(col * 4, col * 4))
    else:
        plt.figure(figsize=figsize)
        
    for i, j in enumerate(img):
        plt.subplot(1, col, i + 1)
        plt.axis("off")
        
        if title != None:
            plt.title(title[i])
        if cmap != None and cmap[i] != "":
            plt.imshow(j, cmap=cmap[i])
        else:
            imshow(j)

## Chapter 2

In [9]:
def imread(fname):
    return io.imread(os.path.join("..", "Image", "read", fname))

In [10]:
def imsave(fname, image):
    io.imsave(os.path.join("..", "Image", "save", fname), image)

## Chapter 3

In [11]:
def spatial_resolution(image, scale):
    return rescale(rescale(image, 1 / scale), scale, order=0)

In [12]:
def grayslice(image, n):
    image = img_as_ubyte(image)
    v = 256 // n
    return image // v * v   

##  Chapter 4

In [13]:
def imhist(image, equal=False):
    if equal:
        image = img_as_ubyte(equalize_hist(image))
    f = plt.figure()
    f.show(plt.hist(image.flatten(), bins=256))

## Chapter 5

In [14]:
def unsharp(alpha=0.2):
    A1 = array([[-1, 1, -1], 
                [1, 1, 1], 
                [-1, 1, -1]], dtype=float64)
    A2 = array([[0, -1, 0], 
                [-1, 5, -1], 
                [0, -1, 0]], dtype=float64)
    return (alpha * A1 + A2) / (alpha + 1)

## Chapter 6

In [15]:
ne = array([[1, 1, 0], [1, 1, 0], [0, 0, 0]])
bi = array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 4
bc = array([[1, 4, 6, 4, 1], 
            [4, 16, 24, 16, 4], 
            [6, 24, 35, 24, 6], 
            [4, 16, 24, 16, 4], 
            [1, 4, 6, 4, 1]]) / 64

In [16]:
def zeroint(img):
    r, c = img.shape
    res = zeros((r*2, c*2))
    res[::2, ::2] = img
    return res

In [17]:
def spatial_filtering(img, p, filt):
    for i in range(int(log2(p))):
        img_zi = zeroint(img)
        img_sf = correlate(img_zi, filt, mode="reflect")
    return img_sf

## Chapter 7

In [18]:
def fftformat(F):
    for f in F:
        print("%8.4f %+.4fi" % (f.real, f.imag))

In [32]:
def fftshow(f, type="log"):
    if type == "log":
        return rescale_intensity(np.log(1 + abs(f)), out_range=(0, 1))
    elif type == "abs":
        return rescale_intensity(abs(f), out_range=(0, 1))

In [4]:
def circle_mask(img, type, lh, D=15, n=2, sigma=10):
    r, c = img.shape
    arr = arange(-r / 2, r / 2)
    arc = arange(-c / 2, c / 2)
    x, y = np.meshgrid(arr, arc)
        
    if type == "ideal":        
        if lh == "low":
            return x**2 + y**2 < D**2
        elif lh == "high":
            return x**2 + y**2 > D**2
    elif type == "butterworth":
        if lh == "low":
            return 1 / (1 + (np.sqrt(2) - 1) * ((x**2 + y**2) / D**2)**n)
        elif lh == "high":
            return 1 / (1 + (D**2 / (x**2 + y**2))**n)
    elif type == "gaussian":
        g = np.exp(-(x**2 + y**2) / sigma**2)
        if lh == "low":
            return g / g.max()
        elif lh == "high":
            return 1 - g / g.max()

In [5]:
def fft_filter(img, type, lh, D=15, n=2, sigma=10):
    f = fftshift(fft2(img))
    c = circle_mask(img, type, lh, D, n, sigma)
    fc = f * c
    return fftshow(f), c, fftshow(fc), fftshow(ifft2(fc), "abs")

In [6]:
def fft_inverse(img):
    pass

## Chapter 8

In [16]:
def periodic_noise(img, s=None):    
    if "numpy" not in str(type(s)):
        r, c = img.shape
        x, y = np.mgrid[0:r, 0:c].astype(float64)
        s = np.sin(x / 3 + y / 3) + 1
    return (2 * img_as_float(img) + s / 2) / 3

In [21]:
def outlier_filter(img, D=0.5):
    av = array([[1, 1, 1], 
            [1, 0, 1], 
            [1, 1, 1]]) / 8
    img_av = convolve(img, av)
    r = abs(img - img_av) > D
    return r * img_av + (1 - r) * img

In [22]:
def image_average(img, n):
    x, y = img.shape
    t = zeros((x, y, n))
    for i in range(n):
        t[:, :, i] = random_noise(img, "gaussian")
    return np.mean(t, 2)

In [ ]:
def pseudo_median(x):
    MAXMIN = 0
    MINMAX = 255
    for i in range(len(x) - 2):
        MAXMIN = max(MAXMIN, min(x[i:i+3]))
        MINMAX = min(MINMAX, max(x[i:i+3]))
    return 0.5 * (MAXMIN + MINMAX)

In [1]:
def periodic_filter(img, type="band", k=1):
    r, c = img.shape
    x_mid, y_mid = r // 2, c // 2
    
    f = fftshift(fft2(img))
    f2 = img_as_ubyte(fftshow(f, "abs"))
    f2[x_mid, y_mid] = 0
    x, y = np.where(f2 == f2.max())
    d = np.sqrt((x[0] - x_mid)**2 + (y[0] - y_mid)**2)
    
    if type == "band":
        x, y = np.meshgrid(arange(0, r), arange(0, c))
        z = np.sqrt((x - x_mid)**2 + (y - y_mid)**2)
        br = (z < np.floor(d - k)) | (z > np.ceil(d + k))
        fc = f * br
    elif type == "criss":
        fc = np.copy(f)
        fc[x, :] = 0
        fc[:, y] = 0    
    
    fci = ifft2(fc)
    return fftshow(f), fftshow(fc), fftshow(fci, "abs") 

In [7]:
def fft_inverse(img, c, type="low", D2=15, n2=2, d=0.01):
    f = fftshift(fft2(img_as_ubyte(img)))
    if type == "low":
        c2 = circle_mask(img, "butterworth", "low", D2, n2, 10)
        fb = f / c * c2
    elif type == "con":
        c2 = np.copy(c)
        c2[np.where(c2 < d)] = 1
        fb = f / c2
    return c2, fftshow(ifft2(fb), "abs")

In [15]:
def deblur(img, m, type="con",d=0.02):
    m2 = zeros_like(img, dtype=float64)
    r, c = m.shape
    m2[0:r, 0:c] = m
    mf = fft2(m2)
    
    if type == "div":
        bmi = ifft2(fft2(img) / mf)
        bmu = fftshow(bmi, "abs")
    elif type == "con":        
        mf[np.where(abs(mf) < d)] = 1
        bmi = abs(ifft2(fft2(img) / mf))
        bmu = img_as_ubyte(bmi / bmi.max())
        bmu = rescale_intensity(bmu, in_range=(0, 128))
    return bmu

## Chapter 9

In [10]:
def threshold_adaptive(img, cut):
    r, c = img.shape
    w = c // cut
    starts = range(0, c - 1, w)  
    ends = range(w, c + 1, w)
    z = zeros((r, c))
    for i in range(cut):
        tmp = img[:, starts[i]:ends[i]]
        z[:, starts[i]:ends[i]] = tmp > threshold_otsu(tmp)
    return z